cmd로 gguf파일 다운로드하기

In [ ]:
%%bash
huggingface-cli download \
  heegyu/EEVE-Korean-Instruct-10.8B-v1.0-GGUF \
  ggml-model-Q5_K_M.gguf \
  --local-dir C:/Users/dm705/Study \
  --local-dir-use-symlinks False

In [1]:
# 이래도 api 뜨나..?

# `*.pptx` file에서 텍스트 추출하기.

## 1. 모델 불러오기(EEVE-Korean-10.8B)

In [ ]:
"""
requirements: pip install langchain
"""

from langchain_community.chat_models import ChatOllama   
from langchain_core.output_parsers import StrOutputParser 
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_core.callbacks.manager import CallbackManager
from langchain import LLMChain

llm = ChatOllama(
    model="EEVE-Korean-10.8B:latest",
    callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),
)

## 2. 프롬프트 템플릿 설정

In [ ]:
import torch
if torch.cuda.is_available():
    torch.cuda.set_device(torch.device("cuda:0"))
    torch.cuda.empty_cache()

In [ ]:
from langchain import LLMChain

# 2. 프롬프트 템플릿 만들기
template = """
입력된 텍스트를 문장 단위로 나누어 주세요:
{text}

나뉘어진 문장별로 번호를 매겨 나누어 정리해 주십시오.
결과물 출력 외에 질문이나 답변은 하지 마십시오.
"""
prompt = ChatPromptTemplate.from_template(template)

# 3. 출력 파서 설정하기
output_parser = StrOutputParser()

# 4. LLMChain 만들기
llm_chain = LLMChain(llm=llm, prompt=prompt, output_parser=output_parser)
# chain = llm | prompt | output_parser

# 5. 텍스트 입력하고 실행하기
input_text = """
장애인 업무의 한계를 없애다 상시 근로자 100,016 명 / 장애인 근로자 1,386 명 (1.54%) * 2018.12 월 기준 장애물 없는 생활환경 (Barrier Free) 자체 인증 제도를 각 사업장에 도입 상담심리사를 상주시켜 직무갈등 , 대인관계 등 고충을 해결 업무에 집중할 환경을 마련 장애인 고객의 입장에서 바라본 기술개발에도 적극 참여 장애인은 단순 업무만 해야 하나 ? ‘ 삼성전자 맞춤형 인재 ’ 로 성장하는데 있어서 비장애인과 장애인의 구분은 의미가 없다고 판단 , 직무교육을 거쳐 고용된 장애인은 각 사업장에서 전기 · 전자 , 정보기술 , 기계디자인 분야에서 근무 중 장애인고용 기업 사례 장애인 고용 전문가와 함께 다양한 장애인 고용 대책 마련
"""

# LLMChain 실행하기
result = llm_chain.invoke({"text": input_text})
# result = chain.invoke({"text": input_text})
# print(result)

## 3. 전처리 - 문장 구분하기 (using Ollama)

In [ ]:
import pandas as pd

# 5. CSV 파일 읽고 처리하기
input_csv_path = "extract_text_[PPT] 직장 내 장애인 인식개선 교육 표준교안 PPT(국문)_수정.csv"
df = pd.read_csv(input_csv_path)

# 텍스트 컬럼 이름 지정
text_column = "Text"

# 결과 저장할 리스트
results = []

# 각 텍스트에 대해 처리
for text in df[text_column]:
    result = llm_chain.run({"text": text})
    results.append(result)

# 결과를 데이터프레임에 추가
df["processed_text"] = results

# 결과를 새로운 CSV 파일로 저장
output_csv_path = "output.csv"
df.to_csv(output_csv_path, index=False)

In [ ]:
df.head()

In [ ]:
df.iloc[4]['processed_text']